 # CNN Example with PySNN

In [ ]:
import os
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

from pysnn.network import SNNNetwork
from pysnn.connection import Conv2d, AdaptiveMaxPool2d
from pysnn.neuron import FedeNeuron, Input
from pysnn.learning import FedeSTDP
from pysnn.utils import conv2d_output_shape
from pysnn.datasets import nmnist_train_test

 ## Parameters used during simulation

In [ ]:
# Architecture
n_in = 5
n_hidden1 = 10
n_hidden2 = 10
n_out = 5

# Data
sample_length = 300
num_workers = 0
batch_size = 1

# Neuronal Dynamics
thresh = 0.8
v_rest = 0
alpha_v = 0.2
tau_v = 5
alpha_t = 1.0
tau_t = 5
duration_refrac = 5
dt = 1
delay = 3
n_dynamics = (thresh, v_rest, alpha_v, alpha_t, dt, duration_refrac, tau_v, tau_t)
c_dynamics = (batch_size, dt, delay)
i_dynamics = (dt, alpha_t, tau_t)

# Learning
lr = 0.0001
w_init = 0.5
a = 0.5

 ## Network definition
 The API is mostly the same aas for regular PyTorch. The main differences are that layers are composed of a Neuron and Connection type, and the layer has to be added to the network by calling the add_layer method. Lastly, all objects return both a spike (or activation potential) object and a trace object.

In [ ]:
class Network(SNNNetwork):
    def __init__(self):
        super(Network, self).__init__()

        # Input
        self.input = Input(
            (batch_size, 2, 34, 34), *i_dynamics, update_type="exponential"
        )

        # Layer 1
        self.conv1 = Conv2d(2, 4, 5, (34, 34), *c_dynamics, padding=1, stride=1)
        self.neuron1 = FedeNeuron((batch_size, 4, 32, 32), *n_dynamics)
        self.add_layer("conv1", self.conv1, self.neuron1)

        # Layer 2
        self.conv2 = Conv2d(4, 8, 5, (32, 32), *c_dynamics, padding=1, stride=2)
        self.neuron2 = FedeNeuron((batch_size, 8, 15, 15), *n_dynamics)
        self.add_layer("conv2", self.conv2, self.neuron2)

        # Layer out
        self.conv3 = Conv2d(8, 1, 3, (15, 15), *c_dynamics)
        self.neuron3 = FedeNeuron((batch_size, 1, 13, 13), *n_dynamics)
        self.add_layer("conv3", self.conv3, self.neuron3)

    def forward(self, input):
        x, t = self.input(input)

        # Layer 1
        x, t = self.conv1(x, t)
        x, t = self.neuron1(x, t)

        # Layer 2
        # x = self.pool2(x)
        x, t = self.conv2(x, t)
        x, t = self.neuron2(x, t)

        # Layer out
        x, t = self.conv3(x, t)
        x, t = self.neuron3(x, t)

        return x, t

 ## Dataset
 The dataset is the Neuromorphic NMNIST dataset, obtained from the following website: [NMNIST-link](https://www.garrickorchard.com/datasets/n-mnist)

In [ ]:
root = "nminst"
if os.path.isdir(root):
    train_dataset, test_dataset = nmnist_train_test(root)
else:
    raise NotADirectoryError(
        "Make sure to download the N-MNIST dataset from https://www.garrickorchard.com/datasets/n-mnist and put it in the 'nmnist' folder."
    )

train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
)

 ## Training
 Training is performed on just three images in order to keep computational time low.
 
 Automatically uses GPU if available on system, also shows the possibility of using 16 bit floating point calculations.

In [ ]:
net = Network()
if torch.cuda.is_available():
    device = torch.device("cuda")
    net = net.to(torch.float16).cuda()
else:
    device = torch.device("cpu")
learning_rule = FedeSTDP(net.layer_state_dict(), lr, w_init, a)

# Go over three items
output = []
for _ in range(3):
    out = []
    batch = next(iter(train_dataloader))
    input = batch[0]
    for idx in range(input.shape[-1]):
        x = input[:, :, :, :, idx].to(device)
        y, _ = net(x)
        out.append(y)

        learning_rule.step()

    net.reset_state()
    output.append(torch.stack(out, dim=-1))

print(output[0].shape)